In [1]:
from components.HarvestWorld import HarvestWorld

from importlib import import_module, reload

import ipywidgets as widgets

import matplotlib.pyplot as plt
from IPython.display import clear_output

harvest_world_module = import_module('components.HarvestWorld')
reload(harvest_world_module)

<module 'components.HarvestWorld' from '/home/raul/Escritorio/extra/misis/rl/rl_uniandes/dynamic_actions/components/HarvestWorld.py'>

In [2]:
ASCII_MAP = """
AA              AA
A    Q      Q    A
P   PPPPPPPPPPP  P
"""

harvest_world = HarvestWorld(ASCII_MAP, 
                            regrowth_probs=[0.0, 0.005],
                            num_agents=1)
    

In [ ]:
def update_plot(freq):
    clear_output(wait=True)
    fig, axes = plt.subplots(1, 2, figsize=(20, 15))
    harvest_world.plot_map(axes[0])
    harvest_world.agents[0].plot_knowledge(axes[1])
    harvest_world.advance_timestep()
    plt.show() 
    
freq_slider = widgets.FloatSlider(value=0, min=0, max=10, step=1, description='Frequency:')
out = widgets.interactive_output(update_plot, {'freq': freq_slider})
display(freq_slider, out)

FloatSlider(value=0.0, description='Frequency:', max=10.0, step=1.0)

Output()

In [8]:
harvest_world.agents[0].q_knowledge[(0,15)]

{'left': 0.0, 'right': 0.1, 'down': 0.0}

In [5]:
harvest_world.agents[0].q_knowledge[(1,1)]

{}

In [6]:
harvest_world.agents[0].current_state

(1, 15)